In [1]:
import openmc
import openmc.deplete
import openmc.model

Import geometry, materials, and settings

In [47]:
geometry = openmc.Geometry.from_xml('ref_geometry.xml')
materials = openmc.Materials.from_xml('ref_materials.xml')
settings = openmc.Settings.from_xml('ref_settings.xml')

/home/ooblack/projects/openmc/openmc/mixin.py:67: IDWarning: Another Material instance already exists with id=1.
  warn(msg, IDWarning)
/home/ooblack/projects/openmc/openmc/mixin.py:67: IDWarning: Another Material instance already exists with id=2.
  warn(msg, IDWarning)
/home/ooblack/projects/openmc/openmc/mixin.py:67: IDWarning: Another Material instance already exists with id=3.
  warn(msg, IDWarning)


In [6]:
ref_materials = openmc.Materials.from_xml('ref_materials.xml')

Create tallies

In [3]:
tallies = openmc.Tallies()

tally = openmc.Tally(name='delayed-neutrons')
tally.filters = [openmc.DelayedGroupFilter([1,2,3,4,5,6])]
tally.scores = ['delayed-nu-fission']
tallies.append(tally)

tally = openmc.Tally(name='delayed-neutrons-1g')
tally.filters = [openmc.DelayedGroupFilter([1])]
tally.scores = ['delayed-nu-fission']
tallies.append(tally)

tally = openmc.Tally(name='total-neutrons')
tally.filters = [openmc.MaterialFilter(materials[0])]
tally.scores = ['nu-fission']
tallies.append(tally)

#tally = openmc.Tally(name='neutrons-energy-bin')
#tally.filters = [openmc.EnergyFilter([1e-8,200000000.0])]
#tally.scores = ['nu-fission' 'delayed-nu-fission']
#tallies.append(tally)

tally = openmc.Tally(name='precursor-decay')
tally.filters = [openmc.DelayedGroupFilter([1,2,3,4,5,6])]
tally.scores = ['decay-rate']
tallies.append(tally)

tally = openmc.Tally(name='precursor-decay-1g')
tally.filters = [openmc.DelayedGroupFilter([1])]
tally.scores = ['decay-rate']
tallies.append(tally)

tally = openmc.Tally(name='fission-energy')
tally.filters = [openmc.UniverseFilter(geometry.root_universe)]
tally.scores = ['fission-q-recoverable', 'fission-q-prompt', 'kappa-fission']
tallies.append(tally)

tally = openmc.Tally(name='normalization')
tally.filters = [openmc.UniverseFilter(geometry.root_universe)]
tally.scores = ['heating']
tallies.append(tally)

tallies.export_to_xml()

/home/ooblack/projects/openmc/openmc/mixin.py:67: IDWarning: Another Filter instance already exists with id=1.
  warn(msg, IDWarning)
/home/ooblack/projects/openmc/openmc/mixin.py:67: IDWarning: Another Filter instance already exists with id=2.
  warn(msg, IDWarning)
/home/ooblack/projects/openmc/openmc/mixin.py:67: IDWarning: Another Filter instance already exists with id=6.
  warn(msg, IDWarning)


Create model and depletion parameters

In [46]:
sp.close()

In [48]:
model = openmc.model.Model(geometry=geometry, 
                           materials=materials, 
                           settings=settings,
                           tallies=tallies)
timesteps = [3 * 3600 * 24] # 3 days -> seconds
chain_path = "./chain_simple.xml"
power = 174
operator = openmc.deplete.Operator(model, chain_file=chain_path)
hm_before = operator.heavy_metal
operator_kwargs = {'chain_file': chain_path}
integrator_kwargs = {'power': power}

In [50]:
model.deplete(timesteps, operator_kwargs=operator_kwargs, power=power)

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

       18/1    0.71582    0.79502 +/- 0.02901
       19/1    0.69543    0.78396 +/- 0.02788
       20/1    0.68739    0.77430 +/- 0.02674
       21/1    0.84266    0.78051 +/- 0.02497
       22/1    0.80714    0.78273 +/- 0.02290
       23/1    0.81266    0.78503 +/- 0.02119
       24/1    0.78144    0.78478 +/- 0.01962
       25/1    0.72566    0.78084 +/- 0.01869
       26/1    0.68751    0.77500 +/- 0.01843
       27/1    0.77064    0.77475 +/- 0.01731
       28/1    0.72323    0.77189 +/- 0.01657
       29/1    0.70491    0.76836 +/- 0.01607
       30/1    0.71617    0.76575 +/- 0.01546
       31/1    0.77101    0.76600 +/- 0.01471
       32/1    0.77461    0.76639 +/- 0.01403
       33/1    0.73821    0.76517 +/- 0.01346
       34/1    0.67636    0.76147 +/- 0.01341
       35/1    0.76799    0.76173 +/- 0.01287
       36/1    0.72223    0.76021 +/- 0.01245
       37/1    0.91325    0.76588 +/- 0.01326
       38/1    0.88221    0.77003 +/- 0.01343
       39/1    0.67833    0.76687 

### Process data

In [7]:
results = openmc.deplete.ResultsList.from_hdf5("./depletion_results.h5")


In [ ]:
delayed_neutrons = sp.get_tally(name='delayed-neutrons')
tot_neutrons = sp.get_tally(name='total-neutrons')
precursor_decay = sp.get_tally(name='precursor-decay')
fission_energy = sp.get_tally(name='fission-energy')
normalization = sp.get_tally(name='normalization')
delayed_neutrons_1g = sp.get_tally(name='delayed-neutrons-1g')
precursor_decay_1g = sp.get_tally(name='precursor-decay-1g')

#### Calculating burnup

It seems to be the case that running ``Operatore.deplete()`` will modify the existing ``Materials`` object with the new material compositions. This is handy, but unexpected. Need to investigate more.

Also, when creating a new material from the depletion results (``ResultsList.export_to_materials()``), the ``get_mass`` function returns the incorrect value. May be a units thing

In [39]:
depleted_materials = results.export_to_materials(1)
new_operator = openmc.deplete.Operator(openmc.Model(geometry=geometry,
                                                materials=depleted_materials),
                                   chain_file=chain_path)
hm_after = new_operator.heavy_metal

In [34]:
burnup = power * (timesteps[0] / 3600 * 24) / (hm_before - hm_after) * 1e-3
burnup

530804.3807519117

In [ ]:
materials[0].get_mass()

In [ ]:
depleted_materials[0].get_mass()

### Tallies

In [57]:
sp = openmc.StatePoint('statepoint.50.h5')
delayed_neutrons = sp.get_tally(name='delayed-neutrons')
tot_neutrons = sp.get_tally(name='total-neutrons')
precursor_decay = sp.get_tally(name='precursor-decay')
fission_energy = sp.get_tally(name='fission-energy')
normalization = sp.get_tally(name='normalization')
delayed_neutrons_1g = sp.get_tally(name='delayed-neutrons-1g')
precursor_decay_1g = sp.get_tally(name='precursor-decay-1g')

### Calculating $\beta_{eff}$

In [ ]:
beta = delayed_neutrons/tot_neutrons
beta_eff = beta.get_pandas_dataframe()['mean'].sum()

beta_1g = delayed_neutrons_1g/tot_neutrons
beta_eff_1g = beta_1g.get_pandas_dataframe()['mean'].sum()

print(f'beta_eff: {beta_eff}')
print(f'beta_eff_1g: {beta_eff_1g}')

### Calculating $\lambda_{eff}$

In [ ]:
precursor_lambda = precursor_decay / delayed_neutrons
coeffs = delayed_neutrons/delayed_neutrons.get_pandas_dataframe()['mean'].sum()
lambda_eff = (coeffs * precursor_lambda).get_pandas_dataframe()['mean'].sum()

precursor_lambda_1g = precursor_decay_1g / delayed_neutrons_1g
lambda_eff_1g = precursor_lambda_1g.get_pandas_dataframe()['mean'].sum()

print(f'lambda_eff: {lambda_eff}')
print(f'lambda_eff_1g: {lambda_eff_1g}')

#### Calculating burnup

In [ ]:
normalization = 1.602e-19 * normalization


In [ ]:
operator = openmc.deplete.Operator(model, chain_file='./chain_simple.xml')

In [ ]:
operator.heavy_metal

In [ ]:
model.materials

In [ ]:
operator.heavy_metal

In [ ]:
hm_before

In [ ]:
model.materials